# CoursewareHubの更新

docker-compose.ymlを更新し、イメージをpullして、CoursewareHubのサービスを再起動します。

- サーバーは再起動しません
- docker serviceの削除と再作成を行います
- ユーザーのNotebookサーバーは停止します

> JupyterHubのデータベーススキーマのアップグレードには未対応

# Notebookと環境のBinding

Inventory中のgroup名でBind対象ホストを指示する。

Ansible Inventory中に定義されているグループ( `[グループ名]` のように`[]`で囲まれた定義 )のうち、CoursewareHub構築に使うグループ名を指定します。

In [ ]:
hosts_file = './cwhtest0001_inventory'
target_group = 'cwhtest0001'

# 接続確認
%env ANSIBLE_INVENTORY={hosts_file}
!ansible -m ping {target_group}

masterとするAnsibleのgroup名を指定する。対象が正しいか、Ansible pingモジュールで動作確認する。

In [ ]:
target_hub = 'cwhub_master'

!ansible -m ping {target_hub} -l {target_group}

workerとするnodeはAnsibleのgroup名で定義する。Ansible pingモジュールで動作確認する。

In [ ]:
target_nodes = 'cwhub_nodes'

!ansible -m ping {target_nodes} -l {target_group}

servicenet_ip変数の確認

In [ ]:
!ansible -m debug -a 'msg={{{{servicenet_ip}}}}' {target_hub}
!ansible -m debug -a 'msg={{{{servicenet_ip}}}}' {target_nodes}

# 現在の状態の確認

In [ ]:
!ansible -b -a 'docker stack ps coursewarehub' {target_hub}

In [ ]:
!ansible -b -a 'docker stack services coursewarehub' {target_hub}

# PostgreSQLのパラメータ定義

In [ ]:
db_user = 'jhauth'
db_user

In [ ]:
db_name = 'jupyterhub'
db_name

以下は内部的に利用するPostgresのパスワード。自身でパスワードを決め、以下に入力する。

(再構築で既存のDBが存在している場合、前回と同じものを設定すること)

In [ ]:
from getpass import getpass

db_pass = getpass()
len(db_pass)

# 環境情報

環境情報をAnsibleの変数から取得する

システム構成のバリエーション

1. フェデレーションに参加するSPとして構成しない (`enable_federation=false`)
  * idp-proxyを使用して、idp-proxy経由でフェデレーションに参加する (`auth_fqdn != ''`)
  * ローカルユーザーのみを使用する (`auth_fqdn == ''`)
2. フェデレーションに参加するSPとして構成する  (`enable_federation=false`)


2.の場合は、運用フェデレーションに参加するSPとして申請し、運用フェデレーションのメタデータに登録済みである必要がある 


CoursewareHubのFQDNをAnsibleに定義した変数から取得する。

In [ ]:
import json

msg_stdout = !ansible -m debug -a 'msg={{{{ master_fqdn }}}}' {target_hub} -l {target_group}
master_fqdn = json.loads(''.join([msg_stdout[0].split()[-1]] + msg_stdout[1:]))['msg']
master_fqdn

In [ ]:
msg_stdout = !ansible -m debug -a 'msg={{{{ auth_fqdn }}}}' {target_hub} -l {target_group}
auth_fqdn = json.loads(''.join([msg_stdout[0].split()[-1]] + msg_stdout[1:]))['msg']
auth_fqdn

In [ ]:
msg_stdout = !ansible -m debug -a 'msg={{{{ enable_federation }}}}' {target_hub} -l {target_group}
enable_federation = json.loads(''.join([msg_stdout[0].split()[-1]] + msg_stdout[1:]))['msg']
enable_federation

# 初期ユーザーイメージの準備

ユーザー環境の初期イメージを用意しておく。

In [ ]:
singleuser_image = 'niicloudoperation/notebook'

In [ ]:
!ansible -b -a 'docker pull {singleuser_image}' {target_hub}

初期イメージとして、タグをつけておく

CoursewareHub起動後にレジストリにpushする必要がある。

In [ ]:
!ansible -b -a 'docker tag {singleuser_image} {master_fqdn}:5000/coursewarehub/initial-course-image' {target_hub}

In [ ]:
!ansible -b -a 'docker images' {target_hub}

# 証明書の確認

In [ ]:
default_user = !ansible -a whoami {target_hub}
default_user = default_user[1]
default_user

In [ ]:
certreq_path = '/home/' + default_user + '/certs'

In [ ]:
import os.path

In [ ]:
print('Certificate:')
print(os.path.join(certreq_path, master_fqdn + ".cer"))
print('Private Key:')
print(os.path.join(certreq_path, master_fqdn + ".key"))

In [ ]:
intermediate_certfile = "nii-odca4g7rsa.cer"

In [ ]:
if intermediate_certfile:
    print(os.path.join(certreq_path, intermediate_certfile))

In [ ]:
!ansible -a 'ls -la {os.path.join(certreq_path, master_fqdn + ".key")}' {target_hub} -l {target_group}
!ansible -a 'ls -la {os.path.join(certreq_path, master_fqdn + ".cer")}' {target_hub} -l {target_group}
if intermediate_certfile:
    !ansible -a 'ls -la {os.path.join(certreq_path, intermediate_certfile)}' {target_hub} -l {target_group}

証明書と秘密鍵を確認する。

証明書の内容と、両者に含まれる公開鍵のfingerprintを確認する。

> 実行が完了しない場合は、パスフレーズが設定されている可能性がある

In [ ]:
!ansible -m shell -a 'openssl rsa -in {os.path.join(certreq_path, master_fqdn + ".key")} -pubout | openssl sha1 -c' {target_hub} -l {target_group}

In [ ]:
!ansible -m shell -a 'openssl x509 -noout -in {os.path.join(certreq_path, master_fqdn + ".cer")} -pubkey | openssl sha1 -c' {target_hub} -l {target_group}
!ansible -a 'openssl x509 -text -noout -in {os.path.join(certreq_path, master_fqdn + ".cer")}' {target_hub} -l {target_group}

idp-proxyを利用しない場合は、idp-proxyの証明書は不要

In [ ]:
if not enable_federation and auth_fqdn != '':
    !ansible -a 'ls -la {os.path.join(certreq_path, auth_fqdn + ".cer")}' {target_hub} -l {target_group}

# auth-proxyコンテナの準備

作業ディレクトリを作成

In [ ]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

設定ファイルを生成

In [ ]:
with open(os.path.join(work_dir, 'hub-const.php'), 'w') as f:
    f.write('''<?php
const COURSE_NAME = "";
const HUB_URL = "http://{master_ip}:8000";
const AUTHOR_GROUP_LIST = array ({{{{ cg_groups | map('regex_replace', '(^|$)', "'") | join('\", \"') }}}});
const DB_USER = "{db_user}";
const DB_PASS = "{db_pass}";
const DB_PORT = "5432";
const DB_NAME = "{db_name}";
const DB_HOST = "postgres";
?>'''.format(master_ip='jupyterhub', db_user=db_user, db_pass=db_pass, db_name=db_name))

設定ファイルを転送

In [ ]:
!ansible -b -m template -a 'src={work_dir}/hub-const.php dest=/etc/jupyterhub/hub-const.php'  {target_hub} -l {target_group}

証明書、秘密鍵等を全ノードにデプロイする


In [ ]:
!ansible -b -m file -a 'path=/etc/jupyterhub/nginx/certs state=directory' {target_hub} -l {target_group}
!ansible -b -a 'cp {os.path.join(certreq_path, master_fqdn + ".key")} /etc/jupyterhub/nginx/certs/auth-proxy.key' {target_hub} -l {target_group}
if intermediate_certfile:
    !ansible -b -m shell -a 'cat {os.path.join(certreq_path, master_fqdn + ".cer")} \
      {os.path.join(certreq_path, intermediate_certfile)} \
      > /etc/jupyterhub/nginx/certs/auth-proxy.chained.cer' {target_hub} -l {target_group}
else:
    !ansible -b -m shell -a 'cat {os.path.join(certreq_path, master_fqdn + ".cer")} \
      > /etc/jupyterhub/nginx/certs/auth-proxy.chained.cer' {target_hub}
!ansible -b -m file -a 'path=/etc/jupyterhub/simplesamlphp/cert state=directory' {target_hub} -l {target_group}
!ansible -b -a 'cp {os.path.join(certreq_path, master_fqdn + ".key")} /etc/jupyterhub/simplesamlphp/cert/auth-proxy.key' {target_hub} -l {target_group}
if not enable_federation and auth_fqdn != '':
    !ansible -b -a 'cp {os.path.join(certreq_path, auth_fqdn + ".cer")} /etc/jupyterhub/simplesamlphp/cert/idp-proxy.cer' {target_hub} -l {target_group}
!ansible -b -m shell -a 'cp {os.path.join(certreq_path, master_fqdn + ".cer")} \
    /etc/jupyterhub/simplesamlphp/cert/auth-proxy.cer' {target_hub} -l {target_group}

In [ ]:
!ansible -b -m shell -a 'chdir=/etc/jupyterhub/ rm -f /exchange/config-jupyterhub.tar.gz && \
         tar czvf /exchange/config-jupyterhub.tar.gz .' {target_hub} -l {target_group}
!ansible -b -m file -a 'path=/etc/jupyterhub state=absent' {target_nodes} -l {target_group}
!ansible -b -m file -a 'path=/etc/jupyterhub state=directory' {target_nodes} -l {target_group}
!ansible -b -m shell -a 'chdir=/etc/jupyterhub/ tar zxvf /exchange/config-jupyterhub.tar.gz' {target_nodes} -l {target_group}
!ansible -b -m file -a 'path=/exchange/config-jupyterhub.tar.gz state=absent' {target_hub} -l {target_group}

SimpleSAMLphp cronモジュールのトリガー用secret文字列を生成

In [ ]:
import random
import string
cron_secret = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
cron_secret

# registryコンテナの準備

In [ ]:
registry_pass = ''.join([random.choice("abcdef" + string.digits) for _ in range(32)])
registry_pass

In [ ]:
!ansible -b -m file -a 'path=/jupyter/registry/data state=directory' {target_hub} -l {target_group}

In [ ]:
!ansible -b -m shell \
    -a 'docker run --rm --entrypoint htpasswd httpd:2 -Bbn cwh {registry_pass} > /jupyter/registry/htpasswd' \
    {target_hub} -l {target_group}

# docker-compose.ymlの生成

> *MEMO auth-proxy内でFQDNでProxyをしたくなる箇所があるようなので、container中からhost NICに迂回しないようhost mappingを追加*

あらかじめ定義されたAnsibleの変数から生成します。

まず、テンプレートを用意します。

In [ ]:
import os

docker_compose_j2_path = os.path.join(work_dir, 'docker-compose.yml.j2')

with open(docker_compose_j2_path, 'w') as f:
    f.write('''version: '3.8'
services:
  auth-proxy:
    deploy:
      placement:
        constraints:
        - node.role==manager
      replicas: 1
    depends_on:
      - jupyterhub
    healthcheck:
      test: ["CMD", "curl", "-k", "-f", "https://localhost/php/login.php"]
      interval: 30s
      timeout: 3s
      retries: 3
      start_period: 5s
    environment:
      AUTH_FQDN: "{{ auth_fqdn }}"
      CRON_SECRET: "{{ cron_secret }}"
      HUB_NAME: jupyterhub
      MASTER_FQDN: "{{ master_fqdn }}"
{% if enable_federation %}
      ENABLE_FEDERATION: "yes"
{%- endif %}
{% if enable_test_federation %}      
      ENABLE_TEST_FEDERATION: "yes"
{% endif %}
      CG_FQDN: "{{ cg_fqdn | default('') }}"
      DS_FQDN: "{{ gakunin_ds_hostname | default('') }}"
    extra_hosts:
    - {{ master_fqdn }}:127.0.0.2
    hostname: auth-proxy
    image: "{{ auth_proxy_image }}"
    networks:
    - backend
    ports:
    - 80:80
    - 443:443
    volumes:
    - read_only: true
      source: /etc/jupyterhub/nginx/certs
      target: /etc/nginx/certs
      type: bind
    - read_only: true
      source: /etc/jupyterhub/hub-const.php
      target: /var/www/lib/hub-const.php
      type: bind
    - read_only: true
      source: /etc/jupyterhub/lti/private.key
      target: /var/www/lib/lti/private.key
      type: bind
    - read_only: true
      source: /etc/jupyterhub/lti/lti.json
      target: /var/www/lib/lti/configs/lti.json
      type: bind
    - read_only: true
      source: /etc/jupyterhub/simplesamlphp/metadata/xml
      target: /var/www/simplesamlphp/metadata/xml
      type: bind
    - read_only: true
      source: /etc/jupyterhub/simplesamlphp/cert
      target: /var/www/simplesamlphp/cert
      type: bind
  jupyterhub:
    deploy:
      placement:
        constraints:
        - node.role==manager
      replicas: 1
    depends_on:
      - postgres
      - registry
    environment:
      BACKEND_NETWORK: courseware-backend
      CONCURRENT_SPAWN_LIMIT: "{{ concurrent_spawn_limit | default('20')}}"
{% if cull_server is defined %}
      CULL_SERVER: '{{ cull_server }}'
{% endif %}
{% if cull_server_every is defined %}
      CULL_SERVER_EVERY: '{{ cull_server_every }}'
{% endif %}
{% if cull_server_idle_timeout is defined %}
      CULL_SERVER_IDLE_TIMEOUT: '{{ cull_server_idle_timeout }}'
{% endif %}
{% if cull_server_max_age is defined %}
      CULL_SERVER_MAX_AGE: '{{ cull_server_max_age }}'
{% endif %}
      DEBUG: 'yes'
      POSTGRES_ENV_JPY_PSQL_PASSWORD: "{{ db_pass }}"
      POSTGRES_ENV_JPY_PSQL_USER: {{ db_user }}
      POSTGRES_PORT_5432_TCP_ADDR: 'postgres'
      RESOURCE_ALLOCATION_FILE: /srv/jupyterhub/resource.yaml
      SPAWNER_CONSTRAINTS: node.role==worker
      SPAWNER_HTTP_TIMEOUT: "{{ spawner_http_timeout | default('120') }}"
      SPAWNER_RESTART_MAX_ATTEMPTS: "{{ spawner_restart_max_attempts | default('5') }}"
      SPAWNER_START_TIMEOUT: "{{ spawner_start_timeout | default('300') }}"
{% if jupyterhub_singleuser_app is defined %}
      JUPYTERHUB_SINGLEUSER_APP: '{{ jupyterhub_singleuser_app }}'
{% endif %}
{% if jupyterhub_singleuser_default_url is defined %}
      JUPYTERHUB_SINGLEUSER_DEFAULT_URL: '{{ jupyterhub_singleuser_default_url }}'
{% endif %}
      REGISTRY_HOST: "{{ master_fqdn }}:5000"
      REGISTRY_USER: "cwh"
      REGISTRY_PASSWORD: "{{ registry_pass }}"
    hostname: jupyterhub
    image: {{ jupyterhub_image }}
    networks:
    - backend
    ports:
    - 8000:8000
    - 8081:8081
    volumes:
    - source: /var/run/docker.sock
      target: /var/run/docker.sock
      type: bind
    - source: /var/run/restuser.sock
      target: /var/run/restuser.sock
      type: bind
    - read_only: true
      source: /var/jupyterhub/logo.png
      target: /var/jupyterhub/logo.png
      type: bind
    - read_only: true
      source: /etc/jupyterhub/resource.yaml
      target: /srv/jupyterhub/resource.yaml
      type: bind
    - read_only: true
      source: /etc/jupyterhub/jupyterhub_config.d
      target: /jupyterhub_config.d
      type: bind
    command: >
      /bin/bash -c
      "jupyterhub upgrade-db && exec jupyterhub"
  postgres:
    deploy:
      placement:
        constraints:
        - node.role==manager
      replicas: 1
    environment:
      PGDATA: /var/lib/postgresql/data/pgdata
      POSTGRES_DB: jupyterhub
      POSTGRES_PASSWORD: "{{ db_pass }}"
      POSTGRES_USER: {{ db_user }}
    hostname: postgres
    image: postgres:13
    networks:
    - backend
    volumes:
    - source: /jupyter/psql/data
      target: /var/lib/postgresql/data
      type: bind
    - source: /jupyter/psql/create.sql
      target: /docker-entrypoint-initdb.d/create.sql
      type: bind
  registry:
    deploy:
      placement:
        constraints:
        - node.role==manager
      replicas: 1
    image: registry:2
    networks:
    - backend
    ports:
    - 5000:5000
    environment:
      REGISTRY_HTTP_TLS_CERTIFICATE: /certs/auth-proxy.chained.cer
      REGISTRY_HTTP_TLS_KEY: /certs/auth-proxy.key
      REGISTRY_AUTH: htpasswd
      REGISTRY_AUTH_HTPASSWD_PATH: /auth/htpasswd
      REGISTRY_AUTH_HTPASSWD_REALM: Registry Realm
      REGISTRY_STORAGE_DELETE_ENABLED: 'true'
    volumes:
    - source: /jupyter/registry/data
      target: /var/lib/registry
      type: bind
    - read_only: true
      source: /jupyter/registry/htpasswd
      target: /auth/htpasswd
      type: bind
    - read_only: true
      source: /etc/jupyterhub/nginx/certs
      target: /certs
      type: bind
networks:
  backend:
    external: true
    name: courseware-backend
'''.lstrip())

In [ ]:
!cat {docker_compose_j2_path}

テンプレートの展開、マスターノードへの配備を行います。

In [ ]:
!ansible -CDv -b --extra-vars "db_user={db_user} db_pass={db_pass} cron_secret={cron_secret} registry_pass={registry_pass}" -m template \
    -a 'src={docker_compose_j2_path} dest=/opt/coursewarehub/docker-compose.yml' {target_hub} -l {target_group}

In [ ]:
!ansible -b --extra-vars "db_user={db_user} db_pass={db_pass} cron_secret={cron_secret} registry_pass={registry_pass}" -m template \
    -a 'src={docker_compose_j2_path} dest=/opt/coursewarehub/docker-compose.yml' {target_hub} -l {target_group}

# イメージ更新

In [ ]:
!ansible -b -a 'docker pull {{{{ auth_proxy_image }}}}' {target_hub} -l {target_group}

In [ ]:
!ansible -b -a 'docker pull {{{{ jupyterhub_image }}}}' {target_hub} -l {target_group}

# 再起動

In [ ]:
import time

!ansible -b -a 'docker stack rm coursewarehub' {target_hub} -l {target_group} ;:
!ansible -b -a 'chdir=/opt/coursewarehub docker stack deploy -c docker-compose.yml coursewarehub' {target_hub} -l {target_group}

サービス起動完了のタイミングが揃わないので、すぐには安定しません。

コンテナが何度か再起動されて、安定するまで待ちます。

In [ ]:
time.sleep(60)

60秒以上かかる場合もあるので、何度か確認してください。

すべての `REPLICAS`が、`1/1`になっていることを確認すること。

In [ ]:
!ansible -b -a 'docker stack services coursewarehub' {target_hub}

ログをチェック

ログの順番は、タイムスタンプの順序通りでない場合があります。

In [ ]:
!ansible -b -a 'docker service logs coursewarehub_auth-proxy' {target_hub}

In [ ]:
!ansible -b -a 'docker service logs coursewarehub_jupyterhub' {target_hub}

# registryにログイン

masterノードでdocker loginをしておきます。

repo2dockerがregistryにイメージをpushするときに認証情報を得るために必要となります。

In [ ]:
!ansible -b -a 'docker login -u cwh -p {registry_pass} {master_fqdn}:5000' {target_hub} -l {target_group}

# registryにイメージをpush

「初期ユーザーイメージの準備」で用意した初期イメージをregistryにpushします。

In [ ]:
!ansible -b -a 'docker push {master_fqdn}:5000/coursewarehub/initial-course-image' {target_hub}

初期ユーザーイメージを、デフォルトのイメージとして設定する場合は、以下のセルのコメントアウトを解除して実行する

In [ ]:
# !ansible -b -a 'docker tag {master_fqdn}:5000/coursewarehub/initial-course-image {master_fqdn}:5000/coursewarehub/default-course-image' {target_hub}
# !ansible -b -a 'docker push {master_fqdn}:5000/coursewarehub/default-course-image' {target_hub}

# 起動後の確認

メタデータファイルの存在と、ログの確認

ローカルユーザーのみ使用する場合、メタデータは存在しなくてもかまわない

コンテナが動作するノードを調べて、コンテナ内部のファイルを確認する。

In [ ]:
%%writefile {work_dir}/get_servicenet_ip.yml
---
- become: true
  hosts: all
  tasks:
      - debug: "msg={{inventory_hostname}}={{servicenet_ip}}"


In [ ]:
import re

servicenet_result = {}

msgout = !ansible-playbook -l {target_group} {work_dir}/get_servicenet_ip.yml
msgout = msgout.grep('"msg"')
assert len(msgout) >= 1
for l in msgout:
    m = re.match(r'.*\"msg\": \"(.+)=(.+)\"', l)
    if m:
        servicenet_result[m.group(2)] = m.group(1)
servicenet_result

In [ ]:
auth_proxy_id = !ansible -b -a 'docker service ps coursewarehub_auth-proxy -q' {target_hub} -l {target_group}
auth_proxy_id = auth_proxy_id[1]
auth_proxy_id = auth_proxy_id.strip()
auth_proxy_id

In [ ]:
output = !ansible -b -a 'docker inspect --format "{{% raw %}} {{{{.NodeID}}}} {{{{.Status.ContainerStatus.ContainerID}}}}" {{% endraw %}} {auth_proxy_id}' {target_hub} -l {target_group}
auth_proxy_node_id, auth_proxy_container_id = output[1].split()
(auth_proxy_node_id, auth_proxy_container_id)

In [ ]:
output = !ansible -b -a 'docker node inspect --format "{{% raw %}} {{{{.Status.Addr}}}}" {{% endraw %}} {auth_proxy_node_id}' {target_hub} -l {target_group}
auth_proxy_node_ip = output[1].split()
auth_proxy_node_ip = [servicenet_result[x] for x in auth_proxy_node_ip]
auth_proxy_node_ip

In [ ]:
!ansible -b -a 'docker exec -i {auth_proxy_container_id} find /var/www/simplesamlphp/metadata/' \
    {auth_proxy_node_ip[0]}

In [ ]:
!ansible -b -a 'docker exec -i {auth_proxy_container_id} cat /var/www/simplesamlphp/log/simplesamlphp.log' \
    {auth_proxy_node_ip[0]}

idp-proxyを利用する場合は、そのメタデータが必要

idp-proxyを使用しない場合は、空で問題無い。

In [ ]:
!ansible -b -a 'docker exec -i {auth_proxy_container_id} ls -l /var/www/simplesamlphp/metadata/idp-proxy' \
    {auth_proxy_node_ip[0]}

フェデレーションに直接参加するSPの場合、フェデレーションメタデータが必要。それ以外ではエラーでも問題ない。

In [ ]:
!ansible -b -a 'docker exec -i {auth_proxy_container_id} ls -l /var/www/simplesamlphp/metadata/gakunin-metadata' \
    {auth_proxy_node_ip[0]} ;:

必要なメタデータが無い場合は、手動でメタデータを更新する。以下のページにアクセスする。

In [ ]:
import json

msg_stdout = !ansible -m debug -a 'msg={{{{ master_fqdn }}}}' {target_hub} -l {target_group}
master_fqdn = json.loads(''.join([msg_stdout[0].split()[-1]] + msg_stdout[1:]))['msg']
master_fqdn

In [ ]:
import re
msg_stdout = !ansible -b -a 'docker inspect {auth_proxy_container_id}' {auth_proxy_node_ip[0]}

m = re.match(r'.*\"CRON_SECRET=([0-9a-f]+)\"', msg_stdout.grep('CRON_SECRET')[0])
cron_secret = m.group(1)

In [ ]:
!echo "https://{master_fqdn}/simplesaml/module.php/cron/run/daily/{cron_secret}/xhtml"

再確認

In [ ]:
!ansible -b -a 'docker exec -i {auth_proxy_container_id} ls -l /var/www/simplesamlphp/metadata/idp-proxy' \
    {auth_proxy_node_ip[0]}

In [ ]:
!ansible -b -a 'docker exec -i {auth_proxy_container_id} ls -l /var/www/simplesamlphp/metadata/gakunin-metadata' \
    {auth_proxy_node_ip[0]} ;:

# 後始末

In [ ]:
!rm -rf {work_dir}